In [1]:
import pandas as pd
import io
import requests
import glob
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio
# Allow nested event loops in Jupyter Notebook
nest_asyncio.apply()
from bs4 import BeautifulSoup
import re
import calendar
import locale
import yfinance
# Set the locale to German
import pygsheets
import datawrapper

from anthropic import Anthropic



In [2]:
async def tablefinder(url):
    # Launch a browser instance
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # Navigate to the website
        await page.goto(url)
        await page.wait_for_timeout(10000)
        # Find all tables on the page
        tables = await page.query_selector_all('table')
        print(f"Found {len(tables)} tables on the page.")

        # Extract the HTML content of each table
        table_data = []
        for i, table in enumerate(tables):
            table_html = await table.inner_html()
            table_data.append(table_html)

        # Close the browser
        await browser.close()
        return table_data
    


In [3]:
tables=await tablefinder('https://www.tradecomplianceresourcehub.com/2025/05/05/trump-2-0-tariff-tracker/')

Found 2 tables on the page.


In [4]:
soup = BeautifulSoup(tables[0], 'html.parser')

# Select the table
tr = soup.find_all('tr')
outer_array=[]
for i, row in enumerate(tr[2:]):
    cells = row.find_all(['td', 'th'])
    cell_data = [cell.get_text(strip=True) for cell in cells]  # Extract text content
    print(f"Row {i + 1}: {cell_data}")
    outer_array.append(cell_data)
outer_array

Row 1: ['Algeria', 'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)', '30%', 'All products (see exceptions below)', '', '']
Row 2: ['Angola', 'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)', '32%', 'All products (see exceptions below)', '', '']
Row 3: ['Austria', 'Threatened (Feb. 21, 2025)', 'TBD', 'TBD', 'See also EUDefending American Corporations and Innovators from Overseas Extortion Memo(Feb. 21, 2025)USTR may renew Section 301 investigation to address digital services taxes (DSTs)', '']
Row 4: ['Bangladesh', 'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)', '37%', 'All products (see exceptions below)', '', '']
Row 5: ['Belarus', '', '', '', 'Reciprocal tariff exemption:imports not subject to reciprocal tariffs at this time', '']
Row 6: ['Bosnia and Herzegovina', 'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)', '36%', 'All products (see exceptions below)', '', '']
Row 7: ['Botswana', 'Reciprocal tariff:Delaye

[['Algeria',
  'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)',
  '30%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Angola',
  'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)',
  '32%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Austria',
  'Threatened (Feb. 21, 2025)',
  'TBD',
  'TBD',
  'See also EUDefending American Corporations and Innovators from Overseas Extortion Memo(Feb. 21, 2025)USTR may renew Section 301 investigation to address digital services taxes (DSTs)',
  ''],
 ['Bangladesh',
  'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)',
  '37%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Belarus',
  '',
  '',
  '',
  'Reciprocal tariff exemption:imports not subject to reciprocal tariffs at this time',
  ''],
 ['Bosnia and Herzegovina',
  'Reciprocal tariff:Delayed until July 9 (effective Apr. 10, 2025)',
  '36%',
  'All products (see exceptions below)',
  '',
  ''],
 ['Botswana',
 

In [5]:
country_tariff_data=pd.DataFrame(outer_array, columns=['Country', 'Tariffnotes', 'Tariff', 'Product', 'Duty Rate', 'Notes'])
country_tariff_data

,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
0,Algeria,Reciprocal tariff:Delayed until July 9 (effect...,30%,All products (see exceptions below),,
1,Angola,Reciprocal tariff:Delayed until July 9 (effect...,32%,All products (see exceptions below),,
2,Austria,"Threatened (Feb. 21, 2025)",TBD,TBD,See also EUDefending American Corporations and...,
3,Bangladesh,Reciprocal tariff:Delayed until July 9 (effect...,37%,All products (see exceptions below),,
4,Belarus,,,,Reciprocal tariff exemption:imports not subjec...,
...,...,...,...,...,...,...
62,United Kingdom,"Threatened (Feb. 21, 2025)",TBD,TBD,Defending American Corporations and Innovators...,
63,Venezuela,Reciprocal tariff:Delayed until July 9 (effect...,15%,All products (see exceptions below),,
64,Vietnam,Reciprocal tariff:Delayed until July 9 (effect...,46%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
65,Zambia,Reciprocal tariff:Delayed until July 9 (effect...,17%,All products (see exceptions below),,


In [7]:
soup = BeautifulSoup(tables[1], 'html.parser')

# Select the table
tr = soup.find_all('tr')
outer_array=[]
for i, row in enumerate(tr[2:]):
    cells = row.find_all(['td', 'th'])
    cell_data = [cell.get_text(strip=True) for cell in cells]  # Extract text content
    print(f"Row {i + 1}: {cell_data}")
    outer_array.append(cell_data)
outer_array

Row 1: ['Aluminum', 'Implemented (effective Mar. 12, 2025);increased June 4, 2025', '25% for UK-origin products50% for all other foreign-origin products', 'See HTSUS Chapter 99, Notes 19(g) and 19(i)-(k)Effective Apr. 4:includes beer (2203.00.00, HTSUS) and empty aluminum cans (7612.90.10, HTSUS)', 'See related publications belowReciprocal tariff exception: aluminum and derivative products subject to Section 232 tariffs are not subject to the reciprocal tariffs“Stacking” exception (starting June 4): goods that are also subject to the Section 232 tariffs on automobiles or automobile parts are not subject to the Section 232 tariffs on aluminum and derivative products“Stacking” exception(Mar. 4 – June 4):goods that are also subject to (a) the Section 232 tariffs on automobiles or automobile parts or (b) the tariffs on Canadian- and Mexican-origin goods are not subject to the Section 232 tariffs on aluminum and derivative productsOn or after July 9, 2025, the Secretary of Commerce may adju

[['Aluminum',
  'Implemented (effective Mar. 12, 2025);increased June 4, 2025',
  '25% for UK-origin products50% for all other foreign-origin products',
  'See HTSUS Chapter 99, Notes 19(g) and 19(i)-(k)Effective Apr. 4:includes beer (2203.00.00, HTSUS) and empty aluminum cans (7612.90.10, HTSUS)',
  'See related publications belowReciprocal tariff exception: aluminum and derivative products subject to Section 232 tariffs are not subject to the reciprocal tariffs“Stacking” exception (starting June 4): goods that are also subject to the Section 232 tariffs on automobiles or automobile parts are not subject to the Section 232 tariffs on aluminum and derivative products“Stacking” exception(Mar. 4 – June 4):goods that are also subject to (a) the Section 232 tariffs on automobiles or automobile parts or (b) the tariffs on Canadian- and Mexican-origin goods are not subject to the Section 232 tariffs on aluminum and derivative productsOn or after July 9, 2025, the Secretary of Commerce may ad

In [8]:
product_based_df=pd.DataFrame(outer_array)
product_based_df

,0,1,2,3,4
0,Aluminum,"Implemented (effective Mar. 12, 2025);increase...",25% for UK-origin products50% for all other fo...,"See HTSUS Chapter 99, Notes 19(g) and 19(i)-(k...",See related publications belowReciprocal tarif...
1,Automobiles,"Implemented (effective Apr. 3, 2025)",25%,"See HTSUS Chapter 99, notes 33(a)-(e)",Reciprocal tariff exception:automobiles subjec...
2,Automobile parts,"Implemented (effective May 3, 2025)",25%,"See HTSUS Chapter 99, notes 33(g)-(h)",Reciprocal tariff exception:automobileparts su...
3,Commercial aircraft and jet engines and parts,"Threatened (May 13, 2025)",TBD,TBD,"See related publications belowOn May 1, Commer..."
4,Copper,"Threatened (Feb. 25, 2025)",25%,TBD,Commerce initiated a Section 232 investigation...
5,Integrated circuits,"Threatened (Jan. 31, 2025)",TBD,TBD,See related publications belowSee semiconducto...
6,iPhones,"Threatened (May 23, 2025)",25%,TBD,
7,"Lumber, timber, and derivative products","Threatened (Mar. 3, 2025)",25%,TBD,Commerce initiated a Section 232 investigation...
8,Maritime cargo handling equipment,"Threatened (Apr. 9, 2025)",20–100%100%,"Containers, chassis, and chassis parts (HTSUS ...",As part of the USTR’sSection 301 investigation...
9,Movies,"Threatened (May 4, 2025)",100%,All foreign-produced movies,Trump authorized Commerce and the USTR to init...


In [9]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

#Open the google spreadsheet
sh = gc.open('TrumptrackerRaw')

#Define which sheet to open in the file
wks = sh[2]

wks.set_dataframe(product_based_df,(0,0))


In [10]:
claudekeyfile=pd.read_csv("/Users/halukamaier-borst/Documents/anthropoenic.csv")
claude_key=claudekeyfile.APIKey.iloc[0]

In [11]:
client = Anthropic(
    api_key=claude_key
)

In [12]:
def translate_row(testtext):
    """
    Translates the given text from English to German using the Anthropic API.
    """
    # Call the Anthropic API for translation
    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=500,
        temperature=0.2,
        system="You are a translator with expertise in English and German. You are translating a technical document that is about tariffs that will be imposed. Your task is to translate the bits of text that I give you from English to German",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text":f"{testtext}--> Deutsch"
                    }
                ]
            }
        ]
    )
    reply=message.content[0].text
    germanrow=reply.split('!')
    return germanrow

In [13]:
german_array=[]
for index, row in product_based_df.iterrows():
    string = '!'.join(row.tolist())
    translated_row = translate_row(string)
    german_array.append(translated_row)
german_array

[['Aluminium',
  'Umgesetzt (wirksam ab 12. März 2025); erhöht am 4. Juni 2025',
  '25% für Produkte mit Ursprung im Vereinigten Königreich50% für alle anderen Produkte ausländischen Ursprungs',
  'Siehe HTSUS Kapitel 99, Anmerkungen 19(g) und 19(i)-(k)Wirksam ab 4. April:umfasst Bier (2203.00.00, HTSUS) und leere Aluminiumdosen (7612.90.10, HTSUS)',
  'Siehe verwandte Veröffentlichungen untenAusnahme bei gegenseitigen Zöllen: Aluminium und Derivatprodukte, die den Zöllen nach Abschnitt 232 unterliegen, unterliegen nicht den gegenseitigen Zöllen"Stacking"-Ausnahme (ab 4. Juni): Waren, die auch den Zöllen nach Abschnitt 232 auf Automobile oder Autoteile unterliegen, unterliegen nicht den Zöllen nach Abschnitt 232 auf Aluminium und Derivatprodukte"Stacking"-Ausnahme (4. März - 4. Juni): Waren, die auch (a) den Zöllen nach Abschnitt 232 auf Automobile oder Autoteile oder (b) den Zöllen auf Waren mit Ursprung in Kanada und Mexiko unterliegen, unterliegen nicht den Zöllen nach Abschnitt 232

In [14]:
product_german_df=pd.DataFrame(german_array)
product_german_df

,0,1,2,3,4
0,Aluminium,Umgesetzt (wirksam ab 12. März 2025); erhöht a...,25% für Produkte mit Ursprung im Vereinigten K...,"Siehe HTSUS Kapitel 99, Anmerkungen 19(g) und ...",Siehe verwandte Veröffentlichungen untenAusnah...
1,Kraftfahrzeuge,Umgesetzt (wirksam ab 3. Apr. 2025),25%,"Siehe HTSUS Kapitel 99, Anmerkungen 33(a)-(e)",Ausnahme bei gegenseitigen Zöllen: Kraftfahrze...
2,Automobilteile,Umgesetzt (wirksam ab 3. Mai 2025),25%,"Siehe HTSUS Kapitel 99, Anmerkungen 33(g)-(h)",Ausnahme bei gegenseitigen Zöllen: Automobilte...
3,Verkehrsflugzeuge und Strahltriebwerke und Teile,Angedroht (13. Mai 2025),Noch festzulegen,Noch festzulegen,Siehe verwandte Veröffentlichungen untenAm 1. ...
4,Kupfer,Angedroht (25. Feb. 2025),25%,Noch festzulegen,Das Handelsministerium hat eine Untersuchung n...
5,Integrierte Schaltkreise,Bedroht (31. Jan. 2025),Noch festzulegen,Noch festzulegen,Siehe verwandte Veröffentlichungen untenSiehe ...
6,iPhones,Bedroht (23. Mai 2025),25%,Noch festzulegen,None
7,"Holz, Bauholz und daraus abgeleitete Produkte",Angedroht (3. März 2025),25%,Noch festzulegen,Das Handelsministerium hat eine Untersuchung n...
8,# Ausrüstung für die Handhabung von Seefracht,Angedroht (9. Apr. 2025),20–100%100%,"Container, Fahrgestelle und Fahrgestellteile (...",Im Rahmen der Section 301-Untersuchung des UST...
9,Filme,Angedroht (4. Mai 2025),100%,Alle im Ausland produzierten Filme,Trump ermächtigte das Handelsministerium und d...


In [15]:
country_rename_dict = {
    'Canada': 'Kanada',
    'Mexico': 'Mexiko',
    'China': 'China',
    'European Union': 'Europäische Union',
    'Vietnam': 'Vietnam',
    'Taiwan': 'Taiwan',
    'Japan': 'Japan',
    'India': 'Indien',
    'South Korea': 'Südkorea',
    'Thailand': 'Thailand',
    'Switzerland': 'Schweiz',
    'Indonesia': 'Indonesien',
    'Malaysia': 'Malaysia',
    'Cambodia': 'Kambodscha',
    'United Kingdom': 'Vereinigtes Königreich',
    'South Africa': 'Südafrika',
    'Brazil': 'Brasilien',
    'Bangladesh': 'Bangladesch',
    'Singapore': 'Singapur',
    'Israel': 'Israel',
    'Philippines': 'Philippinen',
    'Chile': 'Chile',
    'Peru': 'Peru',
    'Pakistan': 'Pakistan',
    'Sri Lanka': 'Sri Lanka',
    'Russia': 'Russland',
    
}

In [16]:
tables_at_bbc=await tablefinder("https://flo.uri.sh/visualisation/22576562/embed?auto=1")

Found 1 tables on the page.


In [17]:
soup = BeautifulSoup(tables_at_bbc[0], 'html.parser')

# Select the table
tr = soup.find_all('tr')
outer_array=[]
for i, row in enumerate(tr[1:]):
    cells = row.find_all(['td'])
    index=0
    row_array=[]
    for cell in cells:
        innertext=cell.get_text(strip=True)
        if index==0:
            row_array.append(innertext)
        else:
            justnumbers = re.sub(r'[a-zA-Z]', '', innertext)
            justnumbers = re.sub('%', '', justnumbers)

            justnumbers=justnumbers.strip()
            row_array.append(justnumbers)

        index+=1
    outer_array.append(row_array)

tariffs_bbc=pd.DataFrame(outer_array)

tariffs_bbc.columns=["Country","Import","Bisher","Neu"]
tariffs_bbc=tariffs_bbc.iloc[:13]
tariffs_bbc


,Country,Import,Bisher,Neu
0,European Union,18.5,20,10
1,China,13.4,34,145
2,Japan,4.5,24,10
3,Vietnam,4.2,46,10
4,South Korea,4,25,10
5,Taiwan,3.6,32,10
6,India,2.7,26,10
7,UK,2.1,10,10
8,Switzerland,1.9,31,10
9,Thailand,1.9,36,10


In [18]:
brazil_brics=country_tariff_data[country_tariff_data["Country"].str.contains("BRICS")]
brazil_brics.Country=brazil_brics.Country.str.replace("BRICS2","Brazil")
brazil_brics

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_29820/2381861770.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brazil_brics.Country=brazil_brics.Country.str.replace("BRICS2","Brazil")


,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
7,Brazil,"Threatened (Jan. 30, 2025)",100%,All products,,


In [19]:
eu_european_union=country_tariff_data[country_tariff_data["Country"].str.contains("EU")]
eu_european_union.Country=eu_european_union.Country.str.replace("EU","European Union")
eu_european_union

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_29820/1370830003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eu_european_union.Country=eu_european_union.Country.str.replace("EU","European Union")


,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
20,European Union,Reciprocal tariff:Delayed until July 9 (effect...,20%50%200%,All products (see exceptions below)All product...,"During a cabinet meeting on February 26, Presi...","Delayed until July 14 (effective Apr. 10, 2025..."


In [20]:
uk_united_kingdom=country_tariff_data[country_tariff_data["Country"].str.contains("United Kingdom")]
uk_united_kingdom.Country=uk_united_kingdom.Country.str.replace("United Kingdom","UK")
uk_united_kingdom

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_29820/2130632345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uk_united_kingdom.Country=uk_united_kingdom.Country.str.replace("United Kingdom","UK")


,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
62,UK,"Threatened (Feb. 21, 2025)",TBD,TBD,Defending American Corporations and Innovators...,


In [21]:
country_tariff_expanded=pd.concat([country_tariff_data,brazil_brics,eu_european_union,uk_united_kingdom],axis=0)
country_tariff_expanded

,Country,Tariffnotes,Tariff,Product,Duty Rate,Notes
0,Algeria,Reciprocal tariff:Delayed until July 9 (effect...,30%,All products (see exceptions below),,
1,Angola,Reciprocal tariff:Delayed until July 9 (effect...,32%,All products (see exceptions below),,
2,Austria,"Threatened (Feb. 21, 2025)",TBD,TBD,See also EUDefending American Corporations and...,
3,Bangladesh,Reciprocal tariff:Delayed until July 9 (effect...,37%,All products (see exceptions below),,
4,Belarus,,,,Reciprocal tariff exemption:imports not subjec...,
...,...,...,...,...,...,...
65,Zambia,Reciprocal tariff:Delayed until July 9 (effect...,17%,All products (see exceptions below),,
66,Zimbabwe,Reciprocal tariff:Delayed until July 9 (effect...,18%,All products (see exceptions below),,
7,Brazil,"Threatened (Jan. 30, 2025)",100%,All products,,
20,European Union,Reciprocal tariff:Delayed until July 9 (effect...,20%50%200%,All products (see exceptions below)All product...,"During a cabinet meeting on February 26, Presi...","Delayed until July 14 (effective Apr. 10, 2025..."


In [22]:
country_tariff_top_countries=pd.merge(tariffs_bbc, country_tariff_expanded, how="left", left_on="Country", right_on="Country")
country_tariff_top_countries

,Country,Import,Bisher,Neu,Tariffnotes,Tariff,Product,Duty Rate,Notes
0,European Union,18.5,20,10,Reciprocal tariff:Delayed until July 9 (effect...,20%50%200%,All products (see exceptions below)All product...,"During a cabinet meeting on February 26, Presi...","Delayed until July 14 (effective Apr. 10, 2025..."
1,China,13.4,34,145,Reciprocal tariff:Delayed until Aug. 12 (effec...,34%20%25%,"All products, including Hong Kong- and Macau-o...",See related publications belowSee maritimecarg...,"Effective May 14, 2025:additional 10%ad valore..."
2,Japan,4.5,24,10,Reciprocal tariff:Delayed until July 9 (effect...,24%,All products (see exceptions below),,
3,Vietnam,4.2,46,10,Reciprocal tariff:Delayed until July 9 (effect...,46%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
4,South Korea,4,25,10,Reciprocal tariff:Delayed until July 9 (effect...,26%,All products (see exceptions below),,
5,Taiwan,3.6,32,10,Reciprocal tariff:Delayed until July 9 (effect...,32%,All products (see exceptions below),,
6,India,2.7,26,10,Reciprocal tariff:Delayed until July 9 (effect...,27%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
7,UK,2.1,10,10,"Threatened (Feb. 21, 2025)",TBD,TBD,Defending American Corporations and Innovators...,
8,Switzerland,1.9,31,10,Reciprocal tariff:Delayed until July 9 (effect...,32%,All products (see exceptions below),,
9,Thailand,1.9,36,10,Reciprocal tariff:Delayed until July 9 (effect...,37%,All products (see exceptions below),,


In [23]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

#Open the google spreadsheet
sh = gc.open('TrumptrackerRaw')

#Define which sheet to open in the file
wks = sh[1]

wks.set_dataframe(country_tariff_top_countries,(0,0))


In [24]:
country_tariff_top_countries

,Country,Import,Bisher,Neu,Tariffnotes,Tariff,Product,Duty Rate,Notes
0,European Union,18.5,20,10,Reciprocal tariff:Delayed until July 9 (effect...,20%50%200%,All products (see exceptions below)All product...,"During a cabinet meeting on February 26, Presi...","Delayed until July 14 (effective Apr. 10, 2025..."
1,China,13.4,34,145,Reciprocal tariff:Delayed until Aug. 12 (effec...,34%20%25%,"All products, including Hong Kong- and Macau-o...",See related publications belowSee maritimecarg...,"Effective May 14, 2025:additional 10%ad valore..."
2,Japan,4.5,24,10,Reciprocal tariff:Delayed until July 9 (effect...,24%,All products (see exceptions below),,
3,Vietnam,4.2,46,10,Reciprocal tariff:Delayed until July 9 (effect...,46%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
4,South Korea,4,25,10,Reciprocal tariff:Delayed until July 9 (effect...,26%,All products (see exceptions below),,
5,Taiwan,3.6,32,10,Reciprocal tariff:Delayed until July 9 (effect...,32%,All products (see exceptions below),,
6,India,2.7,26,10,Reciprocal tariff:Delayed until July 9 (effect...,27%25%,All products (see exceptions below)All products,"Purchaser of Venezuelan oil, which may trigger...",
7,UK,2.1,10,10,"Threatened (Feb. 21, 2025)",TBD,TBD,Defending American Corporations and Innovators...,
8,Switzerland,1.9,31,10,Reciprocal tariff:Delayed until July 9 (effect...,32%,All products (see exceptions below),,
9,Thailand,1.9,36,10,Reciprocal tariff:Delayed until July 9 (effect...,37%,All products (see exceptions below),,


In [25]:
german_array=[]
for index, row in country_tariff_top_countries.iterrows():
    row_as_list=row.tolist()
    string = '!'.join(str(x) for x in row_as_list)
    translated_row = translate_row(string)
    german_array.append(translated_row)
german_array

[['Europäische Union',
  '18,5',
  '20',
  '10',
  'Gegenseitiger Zoll:Verzögert bis 9. Juli (wirksam ab 10. April 2025)Androhung zum 1. Juni; verzögert bis 9. Juli (23. Mai 2025)Angedroht (13. März 2025)',
  '20%50%200%',
  'Alle Produkte (siehe Ausnahmen unten)Alle ProdukteAlkoholische Produkte, einschließlich Champagner und Wein',
  'Während einer Kabinettssitzung am 26. Februar sagte Präsident Trump, die Ankündigung werde "sehr bald" erfolgen.Memorandum zum Schutz amerikanischer Unternehmen und Innovatoren vor ausländischer Erpressung (21. Februar 2025)',
  'Verzögert bis 14. Juli (wirksam ab 10. April 2025):zusätzliche Zölle zwischen 4,4% und 50% werden auf Waren im Wert von 8 Milliarden Euro erhoben (siehe Anhänge I und II zur Durchführungsverordnung (EU) 2018/886 und Artikel 1(2) der Durchführungsverordnung (EU) 2020/502)Verzögert bis 14. August (wirksam ab 10. April 2025):25% Wertzölle auf US-amerikanische Waren, die in Anhängen II und III der Durchführungsverordnung (EU) 2025/

In [25]:
country_tariff_top_countries_german=pd.DataFrame(german_array)
country_tariff_top_countries_german

,0,1,2,3,4,5,6,7,8
0,Europäische Union,"18,5",20,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,20%25%200%,Alle Produkte (siehe Ausnahmen unten)Noch zu b...,Während einer Kabinettssitzung am 26. Februar ...,Verzögert bis 14. Juli (wirksam ab 10. April 2...
1,China,13.4,34,145,Gegenseitiger Zoll:Umgesetzt (wirksam ab 9. Ap...,125%34% (ab 12. Aug.)20%,"Alle Produkte, einschließlich Waren mit Urspru...",Siehe verwandte Veröffentlichungen untenSiehe ...,Wirksam ab 14. Mai 2025: zusätzlicher 10% Wert...
2,Japan,4.5,24,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,24%,Alle Produkte (siehe Ausnahmen unten),None,None
3,Vietnam,4.2,46,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,46%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",None
4,Südkorea,4,25,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,26%,Alle Produkte (siehe Ausnahmen unten),None,None
5,Taiwan,3.6,32,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,32%,Alle Produkte (siehe Ausnahmen unten),None,None
6,Indien,"2,7",26,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,27%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",None
7,UK,2.1,10,10,Angedroht (21. Feb. 2025),Noch festzulegen,Noch festzulegen,Memorandum zum Schutz amerikanischer Unternehm...,None
8,Schweiz,1.9,31,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,32%,Alle Produkte (siehe Ausnahmen unten),None,None
9,Thailand,1.9,36,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,37%,Alle Produkte (siehe Ausnahmen unten),None,None


In [ ]:
country_tariff_top_countries_german.columns=["Land","Importvolumen","Bisher","Neu","Status","Zollsatz","Produkte","Beschlossen","Zusatz"]
country_tariff_top_countries_german

,Land,Importvolumen,Bisher,Neu,Status,Zollsatz,Produkte,Beschlossen,Zusatz
0,Europäische Union,"18,5",20,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,20%25%200%,Alle Produkte (siehe Ausnahmen unten)Noch zu b...,Während einer Kabinettssitzung am 26. Februar ...,Verzögert bis 14. Juli (wirksam ab 10. April 2...
1,China,13.4,34,145,Gegenseitiger Zoll:Umgesetzt (wirksam ab 9. Ap...,125%34% (ab 12. Aug.)20%,"Alle Produkte, einschließlich Waren mit Urspru...",Siehe verwandte Veröffentlichungen untenSiehe ...,Wirksam ab 14. Mai 2025: zusätzlicher 10% Wert...
2,Japan,4.5,24,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,24%,Alle Produkte (siehe Ausnahmen unten),None,None
3,Vietnam,4.2,46,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,46%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",None
4,Südkorea,4,25,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,26%,Alle Produkte (siehe Ausnahmen unten),None,None
5,Taiwan,3.6,32,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,32%,Alle Produkte (siehe Ausnahmen unten),None,None
6,Indien,"2,7",26,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,27%25%,Alle Produkte (siehe Ausnahmen unten)Alle Prod...,"Käufer von venezolanischem Öl, was zusätzliche...",None
7,UK,2.1,10,10,Angedroht (21. Feb. 2025),Noch festzulegen,Noch festzulegen,Memorandum zum Schutz amerikanischer Unternehm...,None
8,Schweiz,1.9,31,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,32%,Alle Produkte (siehe Ausnahmen unten),None,None
9,Thailand,1.9,36,10,Gegenseitiger Zoll:Verzögert bis 9. Juli (wirk...,37%,Alle Produkte (siehe Ausnahmen unten),None,None


In [257]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

#Open the google spreadsheet
sh = gc.open('TrumptrackerRaw')

#Define which sheet to open in the file
wks = sh[2]

wks.set_dataframe(country_tariff_top_countries_german,(0,0))


In [207]:
product_german_df

,0,1,2,3,4
0,Aluminium,Umgesetzt (wirksam ab 12. März 2025),25%,"Siehe HTSUS Kapitel 99, Anmerkungen 19(g) und ...",Siehe zugehörige Veröffentlichungen untenAusna...
1,Kraftfahrzeuge,Umgesetzt (wirksam ab 3. Apr. 2025),25%,"Siehe HTSUS Kapitel 99, Anmerkungen 33(a)-(e)",Ausnahme bei gegenseitigen Zöllen: Kraftfahrze...
2,# Automobilteile,Umgesetzt (wirksam ab 3. Mai 2025),25%,"Siehe HTSUS Kapitel 99, Anmerkungen 33(g)-(h)",Ausnahme bei gegenseitigen Zöllen: Automobilte...
3,Kupfer,Angedroht (25. Feb. 2025),25%,Noch festzulegen,Das Handelsministerium hat eine Untersuchung n...
4,Integrierte Schaltkreise,Bedroht (31. Jan. 2025),Noch festzulegen,Noch festzulegen,Siehe verwandte Veröffentlichungen untenSiehe ...
5,"Holz, Bauholz und daraus abgeleitete Produkte",Angedroht (3. März 2025),25%,Noch festzulegen,Das Handelsministerium hat eine Untersuchung n...
6,# Ausrüstung für die Handhabung von Seefracht,Angedroht (9. Apr. 2025),20–100%100%,"Container, Fahrgestelle und Fahrgestellteile (...",\n\nIm Rahmen der Section-301-Untersuchung des...
7,Filme,Angedroht (4. Mai 2025),100%,Alle im Ausland produzierten Filme,Trump ermächtigte das Handelsministerium und d...
8,Öl und Gas,Drohung mit Beginn am 18. Februar 2025 (31. Ja...,Noch festzulegen,Noch festzulegen,
9,"Arzneimittel, pharmazeutische Wirkstoffe und a...",Angedroht (18. Feb. 2025),25% oder höher,Noch festzulegen,Siehe verwandte Veröffentlichungen untenAm 1. ...


In [208]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

#Open the google spreadsheet
sh = gc.open('TrumptrackerRaw')

#Define which sheet to open in the file
wks = sh[4]

wks.set_dataframe(product_german_df,(0,0))


In [209]:

def load_excel_from_url(url):
    """Loads an Excel file from a URL into a pandas DataFrame.

    Args:
        url: The URL of the Excel file.

    Returns:
        A pandas DataFrame containing the data from the Excel file, or None if
        an error occurs.
    """
    try:
        headers = {
               "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
           }
        response = requests.get(url, headers=headers, stream=True) # Make the request with headers
        response.raise_for_status()  # Raise an exception for bad status codes (like 403)

        # Read the content into a BytesIO object for pandas
        excel_data = io.BytesIO(response.content)

        return excel_data
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None


In [210]:
us_trade="https://www.census.gov/foreign-trade/statistics/country/index.html#:~:text=Download%20Full%20Dataset%20(XLSX%20%2D%200.1%20MB)"

us_stats=load_excel_from_url(us_trade)
us_stats

In [211]:
us_stats

In [212]:
url="https://www.census.gov/foreign-trade/statistics/country/ctyseasonal.xlsx"


In [213]:

# Download the file using requests
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Create a BytesIO object from the response content
    excel_file = io.BytesIO(response.content)
    
    # Read the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_file, engine='openpyxl')  # Specify engine if needed
    print(df)
else:
    print(f"Failed to download file. Status code: {response.status_code}")

     year  cty_code               cty_desc          BJAN          BFEB  \
0    2009        20    All other countries -7.688157e+09 -6.061251e+09   
1    2009      3510                 Brazil  5.109941e+08  4.469206e+08   
2    2009        17               CAFTA-DR  3.034300e+08  2.818892e+08   
3    2009      1220                 Canada -1.790673e+09 -1.893448e+09   
4    2009      5700                  China -2.036187e+10 -1.900556e+10   
..    ...       ...                    ...           ...           ...   
389  2025         9  South/Central America  4.257569e+09  4.799653e+09   
390  2025      4419            Switzerland -2.280347e+10 -1.881388e+10   
391  2025      5830                 Taiwan -7.527515e+09 -8.699853e+09   
392  2025      4120         United Kingdom -5.089092e+08  3.364602e+09   
393  2025      5520                Vietnam -1.185779e+10 -1.240867e+10   

             BMAR          BAPR          BMAY          BJUN          BJUL  \
0   -6.221323e+09 -7.484819e+09 -5

In [214]:
df.columns

Index(['year', 'cty_code', 'cty_desc', 'BJAN', 'BFEB', 'BMAR', 'BAPR', 'BMAY',
       'BJUN', 'BJUL', 'BAUG', 'BSEP', 'BOCT', 'BNOV', 'BDEC', 'BQ1', 'BQ2',
       'BQ3', 'BQ4', 'EJAN', 'EFEB', 'EMAR', 'EAPR', 'EMAY', 'EJUN', 'EJUL',
       'EAUG', 'ESEP', 'EOCT', 'ENOV', 'EDEC', 'EQ1', 'EQ2', 'EQ3', 'EQ4',
       'IJAN', 'IFEB', 'IMAR', 'IAPR', 'IMAY', 'IJUN', 'IJUL', 'IAUG', 'ISEP',
       'IOCT', 'INOV', 'IDEC', 'IQ1', 'IQ2', 'IQ3', 'IQ4'],
      dtype='object')

In [215]:
df[df.cty_desc=="China"]

,year,cty_code,cty_desc,BJAN,BFEB,BMAR,BAPR,BMAY,BJUN,BJUL,...,IJUL,IAUG,ISEP,IOCT,INOV,IDEC,IQ1,IQ2,IQ3,IQ4
4,2009,5700,China,-2.036187e+10,-1.900556e+10,-2.037261e+10,-1.842612e+10,-1.840727e+10,-1.757020e+10,-1.826092e+10,...,2.372623e+10,2.377449e+10,2.462898e+10,2.558478e+10,2.595788e+10,2.721828e+10,7.435777e+10,7.112548e+10,7.212970e+10,7.876093e+10
22,2010,5700,China,-2.029652e+10,-2.070019e+10,-2.174464e+10,-2.117937e+10,-2.301374e+10,-2.482953e+10,-2.343214e+10,...,3.112880e+10,3.213780e+10,3.171575e+10,3.155222e+10,3.197177e+10,3.198457e+10,8.426605e+10,9.019568e+10,9.498235e+10,9.550855e+10
40,2011,5700,China,-2.378782e+10,-2.508258e+10,-2.324977e+10,-2.348492e+10,-2.478785e+10,-2.487293e+10,-2.485315e+10,...,3.342016e+10,3.350025e+10,3.316765e+10,3.426471e+10,3.409418e+10,3.432169e+10,9.838606e+10,9.821652e+10,1.000881e+11,1.026806e+11
58,2012,5700,China,-2.545280e+10,-2.470782e+10,-2.812206e+10,-2.638355e+10,-2.488227e+10,-2.607179e+10,-2.649391e+10,...,3.568304e+10,3.459547e+10,3.561117e+10,3.547865e+10,3.662270e+10,3.691888e+10,1.053582e+11,1.053510e+11,1.058897e+11,1.090202e+11
76,2013,5700,China,-2.835521e+10,-2.886388e+10,-2.405719e+10,-2.508820e+10,-2.684325e+10,-2.615814e+10,-2.714464e+10,...,3.646774e+10,3.693936e+10,3.684751e+10,3.702828e+10,3.742276e+10,3.833821e+10,1.102530e+11,1.071331e+11,1.102546e+11,1.127893e+11
96,2014,5700,China,-2.650031e+10,-2.756293e+10,-2.735816e+10,-2.892931e+10,-2.803673e+10,-2.919684e+10,-2.848447e+10,...,3.859166e+10,3.891895e+10,4.017323e+10,4.052143e+10,4.050391e+10,4.010795e+10,1.134893e+11,1.161684e+11,1.176838e+11,1.211333e+11
116,2015,5700,China,-2.991712e+10,-2.692465e+10,-3.983263e+10,-2.889481e+10,-3.020138e+10,-2.975705e+10,-2.947784e+10,...,3.963844e+10,4.164453e+10,4.114561e+10,3.955797e+10,3.929929e+10,3.746784e+10,1.258558e+11,1.185922e+11,1.224286e+11,1.163251e+11
136,2016,5700,China,-2.979297e+10,-3.153861e+10,-2.762581e+10,-2.787307e+10,-2.868884e+10,-2.854071e+10,-2.901581e+10,...,3.886216e+10,3.912173e+10,3.826001e+10,3.910617e+10,3.895725e+10,3.895993e+10,1.157574e+11,1.133953e+11,1.162439e+11,1.170233e+11
157,2017,5700,China,-2.864976e+10,-3.005331e+10,-3.141909e+10,-3.312414e+10,-3.103078e+10,-3.153944e+10,-3.132774e+10,...,4.171225e+10,4.151103e+10,4.129575e+10,4.240641e+10,4.434193e+10,4.511047e+10,1.220411e+11,1.267461e+11,1.245190e+11,1.318588e+11
184,2018,5700,China,-3.516939e+10,-3.502248e+10,-3.348426e+10,-3.332531e+10,-3.322285e+10,-3.309945e+10,-3.388871e+10,...,4.433528e+10,4.384805e+10,4.600810e+10,4.483239e+10,4.291889e+10,4.581347e+10,1.382721e+11,1.324859e+11,1.341914e+11,1.335648e+11


In [216]:
base_col=["year","cty_code","cty_desc"]
export_cols=[]
import_cols=[]
for col in df.columns:
    if col[0]=="E":
        export_cols.append(col)
    elif col[0]=="I":
        import_cols.append(col)
base_col

['year', 'cty_code', 'cty_desc']

In [217]:
base_export_cols=base_col+export_cols
base_import_cols=base_col+import_cols
base_import_cols

['year',
 'cty_code',
 'cty_desc',
 'IJAN',
 'IFEB',
 'IMAR',
 'IAPR',
 'IMAY',
 'IJUN',
 'IJUL',
 'IAUG',
 'ISEP',
 'IOCT',
 'INOV',
 'IDEC',
 'IQ1',
 'IQ2',
 'IQ3',
 'IQ4']

In [218]:
export_df=df[base_export_cols]
export_df

,year,cty_code,cty_desc,EJAN,EFEB,EMAR,EAPR,EMAY,EJUN,EJUL,EAUG,ESEP,EOCT,ENOV,EDEC,EQ1,EQ2,EQ3,EQ4
0,2009,20,All other countries,3.092714e+10,3.222777e+10,3.163669e+10,2.979531e+10,3.143728e+10,3.199560e+10,3.202405e+10,3.228728e+10,3.320518e+10,3.501005e+10,3.566379e+10,3.594004e+10,9.479160e+10,9.322820e+10,9.751652e+10,1.066139e+11
1,2009,3510,Brazil,2.310141e+09,2.109861e+09,1.955903e+09,1.938554e+09,2.024336e+09,2.075956e+09,2.122650e+09,2.094565e+09,2.247110e+09,2.364203e+09,2.303859e+09,2.548317e+09,6.375905e+09,6.038846e+09,6.464325e+09,7.216379e+09
2,2009,17,CAFTA-DR,1.594814e+09,1.655722e+09,1.585226e+09,1.413914e+09,1.544395e+09,1.701161e+09,1.627044e+09,1.669154e+09,1.683253e+09,1.745002e+09,1.868941e+09,1.855674e+09,4.835761e+09,4.659470e+09,4.979451e+09,5.469618e+09
3,2009,1220,Canada,1.607276e+10,1.658391e+10,1.551450e+10,1.558492e+10,1.555190e+10,1.559846e+10,1.750951e+10,1.793495e+10,1.800158e+10,1.796037e+10,1.903583e+10,1.930928e+10,4.817117e+10,4.673528e+10,5.344604e+10,5.630547e+10
4,2009,5700,China,4.529859e+09,4.900156e+09,5.187716e+09,5.489906e+09,5.423565e+09,5.808423e+09,5.465313e+09,5.697671e+09,6.215815e+09,6.313081e+09,6.972480e+09,7.492694e+09,1.461773e+10,1.672189e+10,1.737880e+10,2.077826e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,2025,9,South/Central America,1.797870e+10,1.845261e+10,1.794573e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.437704e+10,NaN,NaN,NaN
390,2025,4419,Switzerland,1.753791e+09,2.486928e+09,3.542400e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.783120e+09,NaN,NaN,NaN
391,2025,5830,Taiwan,3.618026e+09,4.033151e+09,4.266353e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.191753e+10,NaN,NaN,NaN
392,2025,4120,United Kingdom,6.208916e+09,9.466578e+09,8.139514e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.381501e+10,NaN,NaN,NaN


In [219]:
df

,year,cty_code,cty_desc,BJAN,BFEB,BMAR,BAPR,BMAY,BJUN,BJUL,...,IJUL,IAUG,ISEP,IOCT,INOV,IDEC,IQ1,IQ2,IQ3,IQ4
0,2009,20,All other countries,-7.688157e+09,-6.061251e+09,-6.221323e+09,-7.484819e+09,-5.537787e+09,-6.201629e+09,-8.128920e+09,...,4.015297e+10,3.968960e+10,4.240630e+10,4.296826e+10,4.423691e+10,4.572743e+10,1.147623e+11,1.124524e+11,1.222489e+11,1.329326e+11
1,2009,3510,Brazil,5.109941e+08,4.469206e+08,3.447423e+08,5.177740e+08,4.190184e+08,4.636909e+08,4.404457e+08,...,1.682204e+09,1.613042e+09,1.690305e+09,1.610670e+09,1.983716e+09,1.778059e+09,5.073248e+09,4.638363e+09,4.985551e+09,5.372444e+09
2,2009,17,CAFTA-DR,3.034300e+08,2.818892e+08,2.177867e+08,6.088876e+07,1.325944e+08,1.921538e+08,-4.636460e+07,...,1.673409e+09,1.675474e+09,1.685764e+09,1.791865e+09,1.882657e+09,1.826492e+09,4.032656e+09,4.273833e+09,5.034646e+09,5.501014e+09
3,2009,1220,Canada,-1.790673e+09,-1.893448e+09,-1.384375e+09,-1.091626e+09,-8.162940e+08,-1.853868e+09,-2.322835e+09,...,1.983234e+10,1.945058e+10,1.978832e+10,2.060733e+10,2.105888e+10,2.177427e+10,5.323966e+10,5.049706e+10,5.907124e+10,6.344048e+10
4,2009,5700,China,-2.036187e+10,-1.900556e+10,-2.037261e+10,-1.842612e+10,-1.840727e+10,-1.757020e+10,-1.826092e+10,...,2.372623e+10,2.377449e+10,2.462898e+10,2.558478e+10,2.595788e+10,2.721828e+10,7.435777e+10,7.112548e+10,7.212970e+10,7.876093e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,2025,9,South/Central America,4.257569e+09,4.799653e+09,3.233067e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.208675e+10,NaN,NaN,NaN
390,2025,4419,Switzerland,-2.280347e+10,-1.881388e+10,-1.473504e+10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.413551e+10,NaN,NaN,NaN
391,2025,5830,Taiwan,-7.527515e+09,-8.699853e+09,-8.748659e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.689356e+10,NaN,NaN,NaN
392,2025,4120,United Kingdom,-5.089092e+08,3.364602e+09,1.172544e+09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.978677e+10,NaN,NaN,NaN


In [222]:
import locale

# Set the locale to English (United States)
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

'en_US.UTF-8'

In [223]:
def df_for_country(cols,country):    
    df_country=df[df["cty_desc"]==country]
    filtered_entries = [entry for entry in cols if 'Q' not in entry ]
    df_long_country=pd.melt(df_country, id_vars=['year'], value_vars=filtered_entries)
    df_long_country["month"]=df_long_country.variable.str[1:]
    df_long_country["date"]=df_long_country["month"]+"-"+df_long_country["year"].astype(str)
    df_long_country["date"] = pd.to_datetime(df_long_country["date"], format="%b-%Y")
    df_long_country.sort_values(by="date", ascending=True, inplace=True)
    df_long_red=df_long_country[["date","value"]]
    

    return(df_long_red)

In [224]:
china_df_export=df_for_country(export_cols,"China")
china_df_export

,date,value
0,2009-01-01,4.529859e+09
17,2009-02-01,4.900156e+09
34,2009-03-01,5.187716e+09
51,2009-04-01,5.489906e+09
68,2009-05-01,5.423565e+09
...,...,...
135,2025-08-01,NaN
152,2025-09-01,NaN
169,2025-10-01,NaN
186,2025-11-01,NaN


In [225]:
country="China"

In [226]:
def comp_calc(country):
    locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')  # For Linux/Mac

    df_sorted_import=df_for_country(import_cols,country)
    df_sorted_export=df_for_country(export_cols,country)
    df_sorted_import_export=pd.merge(df_sorted_export,df_sorted_import, on="date", how="outer",suffixes=("_export", "_import"))
    df_sorted_import_export["diff"]=df_sorted_import_export["value_export"]-df_sorted_import_export["value_import"]
    last_years=df_sorted_import_export[df_sorted_import_export["date"]>"2021-01-01"]
    last_years=last_years.dropna().reset_index(drop=True)
    last_years
    comparison_df=last_years[(last_years["date"]>"01-01-2021")&(last_years["date"]<"01-01-2025")]
    under_trump=last_years[(last_years["date"]>="01-01-2025")]
    array=[]
    for index, row in under_trump.iloc[-3:].iterrows():
        print(f"Date: {row['date']}, Export: {row['value_export']}, Import: {row['value_import']}, Import: {row['diff']}")
        filtered_df = comparison_df[(comparison_df['date'].dt.month == row['date'].month) & (comparison_df['date'].dt.day == row['date'].day)]
        locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')

        month_name = calendar.month_name[row['date'].month]

        inner_array=[month_name]
        for col in filtered_df.columns[1:]:

            value_mean=filtered_df[col].mean()
            relative_perc=row[col]/value_mean
            diff_perc_=(relative_perc-1)*100
            inner_array.append(diff_perc_)

  

        array.append(inner_array)
    comp_df=pd.DataFrame(array,columns=["date","relative_export","relative_import","relative_diff"])
    comp_df=comp_df.T
    comp_df.columns=comp_df.iloc[0]
    comp_df=comp_df.iloc[1:-1]
    return last_years, comp_df


In [227]:
comp_objects=comp_calc("China")
comp_objects[1]

Date: 2025-01-01 00:00:00, Export: 10516375743.994307, Import: 40207460675.73614, Import: -29691084931.74183
Date: 2025-02-01 00:00:00, Export: 11785128152.782928, Import: 38370735571.99087, Import: -26585607419.20794
Date: 2025-03-01 00:00:00, Export: 10818179154.489431, Import: 35622384923.487045, Import: -24804205768.997612


date,Januar,Februar,März
relative_export,-19.809923,-6.416632,-14.470708
relative_import,0.566269,-5.973925,-19.387994


In [228]:
#Open the google spreadsheet
sh = gc.open('TrumptrackerRaw')

#Define which sheet to open in the file
wks = sh[7]

#Get the data from the Sheet into python as DF
#read = wks.get_as_df()
#read
wks.set_dataframe(comp_objects[1].reset_index(),(0,0))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygsheets/worksheet.py:1431: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(nan)


In [229]:
comp_objects[0]

,date,value_export,value_import,diff
0,2021-02-01,1.051433e+10,3.936298e+10,-2.884866e+10
1,2021-03-01,1.228646e+10,4.725738e+10,-3.497092e+10
2,2021-04-01,1.278247e+10,4.193920e+10,-2.915673e+10
3,2021-05-01,1.269632e+10,3.904306e+10,-2.634675e+10
4,2021-06-01,1.296373e+10,3.915807e+10,-2.619434e+10
5,2021-07-01,1.264594e+10,3.846359e+10,-2.581765e+10
6,2021-08-01,1.166424e+10,3.996308e+10,-2.829884e+10
7,2021-09-01,1.164970e+10,4.200119e+10,-3.035149e+10
8,2021-10-01,1.394129e+10,4.333183e+10,-2.939054e+10
9,2021-11-01,1.339007e+10,4.502138e+10,-3.163131e+10


In [230]:
#comp_df_long=pd.melt(comp_objects[1], id_vars=['date'], value_vars=["relative_export","relative_import"])
#comp_df_long["start"]=0
#comp_df_long


In [231]:
#comp_df_long[["date","start","value","variable"]].to_csv("longform.csv",index=False)

In [232]:
comp_objects=comp_calc("Germany")
comp_objects[1]

Date: 2025-01-01 00:00:00, Export: 6516479823.65365, Import: 14161634467.67601, Import: -7645154644.02236
Date: 2025-02-01 00:00:00, Export: 6259626950.905817, Import: 14338296110.721859, Import: -8078669159.816042
Date: 2025-03-01 00:00:00, Export: 7322458471.919997, Import: 14775112265.512352, Import: -7452653793.592355


date,Januar,Februar,März
relative_export,-2.277303,7.167691,26.012199
relative_import,11.726762,16.744229,21.943458


In [233]:
#Open the google spreadsheet
sh = gc.open('TrumptrackerRaw')

#Define which sheet to open in the file
wks = sh[8]

#Get the data from the Sheet into python as DF
#read = wks.get_as_df()
#read
wks.set_dataframe(comp_objects[1].reset_index(),(0,0))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygsheets/worksheet.py:1431: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(nan)


In [234]:
comp_objects[0]

,date,value_export,value_import,diff
0,2021-02-01,5.270868e+09,1.063668e+10,-5.365808e+09
1,2021-03-01,5.081165e+09,1.043641e+10,-5.355243e+09
2,2021-04-01,5.489055e+09,1.071546e+10,-5.226407e+09
3,2021-05-01,5.403539e+09,1.115457e+10,-5.751030e+09
4,2021-06-01,5.870624e+09,1.189320e+10,-6.022579e+09
5,2021-07-01,5.445644e+09,1.106794e+10,-5.622294e+09
6,2021-08-01,5.638345e+09,1.192524e+10,-6.286894e+09
7,2021-09-01,5.308366e+09,1.113220e+10,-5.823834e+09
8,2021-10-01,5.690471e+09,1.124328e+10,-5.552810e+09
9,2021-11-01,5.598455e+09,1.199799e+10,-6.399530e+09


In [235]:
comp_objects[1].to_csv("germany_relative.csv",index=False)

In [236]:
df=pd.DataFrame()

In [237]:

shanghaicomposite = yfinance.download (tickers = "000001.SS", start = "2024-11-04", 
                              interval = "1d")
shanghaicomposite

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,000001.SS,000001.SS,000001.SS,000001.SS,000001.SS
Date,,,,,
2024-11-04,3310.207031,3310.207031,3263.842041,3275.678955,665100
2024-11-05,3386.987061,3386.987061,3302.047119,3306.810059,821100
2024-11-06,3383.805908,3420.999023,3367.081055,3395.221924,895900
2024-11-07,3470.660889,3470.660889,3356.258057,3359.985107,959600
2024-11-08,3452.297119,3509.818115,3443.193115,3502.133057,964000
...,...,...,...,...,...
2025-05-06,3316.114014,3316.447998,3286.989014,3295.250000,474400
2025-05-07,3342.665039,3356.687988,3324.809082,3354.974121,511200


In [238]:

dax40 = yfinance.download (tickers = "^GDAXI", start = "2024-11-04", 
                              interval = "1d")
dax40

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^GDAXI,^GDAXI,^GDAXI,^GDAXI,^GDAXI
Date,,,,,
2024-11-04,19147.849609,19286.039062,19147.759766,19231.859375,43111600
2024-11-05,19256.269531,19263.140625,19104.589844,19170.150391,47179800
2024-11-06,19039.310547,19563.970703,19007.880859,19360.750000,112726700
2024-11-07,19362.519531,19437.880859,19148.789062,19157.220703,83268900
2024-11-08,19215.480469,19411.599609,19146.529297,19411.589844,69792500
...,...,...,...,...,...
2025-05-06,23249.650391,23389.859375,22861.400391,23387.349609,67251500
2025-05-07,23115.960938,23300.880859,23100.929688,23252.949219,66591700


In [239]:
sandp500 = yfinance.download (tickers = "^GSPC", start = "2024-11-04", 
                              interval = "1d")
sandp500

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2024-11-04,5712.689941,5741.430176,5696.509766,5725.149902,3602060000
2024-11-05,5782.759766,5783.439941,5722.100098,5722.430176,3768310000
2024-11-06,5929.040039,5936.140137,5864.890137,5864.890137,6329530000
2024-11-07,5973.100098,5983.839844,5947.209961,5947.209961,4925740000
2024-11-08,5995.540039,6012.450195,5976.759766,5976.759766,4666740000
...,...,...,...,...,...
2025-05-06,5606.910156,5649.580078,5586.040039,5605.870117,4717260000
2025-05-07,5631.279785,5654.729980,5578.640137,5614.180176,4987440000


In [240]:
def reduce_index(df,indexname):
    index_red=pd.DataFrame()
    df["Date"]=df.index
    index_red["Date"]=df["Date"]

    index_red[indexname]=df[df.columns[0]]

    index_red=index_red.reset_index(drop=True)
    return index_red


In [241]:
sandp500=reduce_index(sandp500,"S&P500")
dax40=reduce_index(dax40,"DAX40")
shanghaicomposite=reduce_index(shanghaicomposite,"SHANGHAI")


In [242]:
sandp500_dax40=pd.merge(sandp500, dax40, on="Date", how="outer")
sandp500_dax40=sandp500_dax40.fillna(method="ffill")
sandp500_dax40

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_1040/2970213323.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sandp500_dax40=sandp500_dax40.fillna(method="ffill")


,Date,S&P500,DAX40
0,2024-11-04,5712.689941,19147.849609
1,2024-11-05,5782.759766,19256.269531
2,2024-11-06,5929.040039,19039.310547
3,2024-11-07,5973.100098,19362.519531
4,2024-11-08,5995.540039,19215.480469
...,...,...,...
128,2025-05-06,5606.910156,23249.650391
129,2025-05-07,5631.279785,23115.960938
130,2025-05-08,5663.939941,23352.689453
131,2025-05-09,5659.910156,23499.320312


In [243]:
sandp500_dax40_shanghai=pd.merge(sandp500_dax40, shanghaicomposite, on="Date", how="outer")
sandp500_dax40_shanghai=sandp500_dax40_shanghai.fillna(method="ffill")
sandp500_dax40_shanghai

/var/folders/m2/_syds9sn2bjctwpf6p3_w19r0000gn/T/ipykernel_1040/2611133901.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sandp500_dax40_shanghai=sandp500_dax40_shanghai.fillna(method="ffill")


,Date,S&P500,DAX40,SHANGHAI
0,2024-11-04,5712.689941,19147.849609,3310.207031
1,2024-11-05,5782.759766,19256.269531,3386.987061
2,2024-11-06,5929.040039,19039.310547,3383.805908
3,2024-11-07,5973.100098,19362.519531,3470.660889
4,2024-11-08,5995.540039,19215.480469,3452.297119
...,...,...,...,...
131,2025-05-07,5631.279785,23115.960938,3342.665039
132,2025-05-08,5663.939941,23352.689453,3351.996094
133,2025-05-09,5659.910156,23499.320312,3341.999023
134,2025-05-12,5844.189941,23566.539062,3341.999023


In [244]:
since_inauguration=sandp500_dax40_shanghai[sandp500_dax40_shanghai["Date"]>="2024-11-04"]
since_inauguration

,Date,S&P500,DAX40,SHANGHAI
0,2024-11-04,5712.689941,19147.849609,3310.207031
1,2024-11-05,5782.759766,19256.269531,3386.987061
2,2024-11-06,5929.040039,19039.310547,3383.805908
3,2024-11-07,5973.100098,19362.519531,3470.660889
4,2024-11-08,5995.540039,19215.480469,3452.297119
...,...,...,...,...
131,2025-05-07,5631.279785,23115.960938,3342.665039
132,2025-05-08,5663.939941,23352.689453,3351.996094
133,2025-05-09,5659.910156,23499.320312,3341.999023
134,2025-05-12,5844.189941,23566.539062,3341.999023


In [245]:
index_name=since_inauguration.columns[1]
index_name

'S&P500'

In [246]:
def converter_perc(index_name):
    df_perc=pd.DataFrame()
    df_perc["Date"]=since_inauguration["Date"]
    col_important=since_inauguration[index_name]
    base_value=col_important.iloc[0]
    df_perc[index_name]=(col_important-base_value)*100/base_value
    return(df_perc)

In [247]:

for col in since_inauguration.columns[1:]:
    df_perc=converter_perc(col)
    since_inauguration=pd.merge(since_inauguration,df_perc,on="Date",how="outer",suffixes=["_abs","_perc"])
since_inauguration 

,Date,S&P500_abs,DAX40_abs,SHANGHAI_abs,S&P500_perc,DAX40_perc,SHANGHAI_perc
0,2024-11-04,5712.689941,19147.849609,3310.207031,0.000000,0.000000,0.000000
1,2024-11-05,5782.759766,19256.269531,3386.987061,1.226564,0.566225,2.319493
2,2024-11-06,5929.040039,19039.310547,3383.805908,3.787184,-0.566847,2.223392
3,2024-11-07,5973.100098,19362.519531,3470.660889,4.558451,1.121118,4.847245
4,2024-11-08,5995.540039,19215.480469,3452.297119,4.951259,0.353203,4.292483
...,...,...,...,...,...,...,...
131,2025-05-07,5631.279785,23115.960938,3342.665039,-1.425076,20.723535,0.980543
132,2025-05-08,5663.939941,23352.689453,3351.996094,-0.853363,21.959854,1.262430
133,2025-05-09,5659.910156,23499.320312,3341.999023,-0.923904,22.725637,0.960423
134,2025-05-12,5844.189941,23566.539062,3341.999023,2.301893,23.076688,0.960423


In [248]:
since_inauguration.to_csv("since_inauguration.csv",index=False)

In [249]:
#authorization
gc = pygsheets.authorize(service_file='/Users/halukamaier-borst/Documents/trumptracker-459020-a1e8634a9252.json')

gc

In [250]:
#Open the google spreadsheet
sh = gc.open('TrumptrackerRaw')

#Define which sheet to open in the file
wks = sh[0]

#Get the data from the Sheet into python as DF
read = wks.get_as_df()
read

,Date,S&P500_abs,DAX40_abs,SHANGHAI_abs,S&P500_perc,DAX40_perc,SHANGHAI_perc
0,2024-11-04,5712.689941,19147.84961,3310.207031,0.000000,0.000000,0.000000
1,2024-11-05,5782.759766,19256.26953,3386.987061,1.226564,0.566225,2.319493
2,2024-11-06,5929.040039,19039.31055,3383.805908,3.787184,-0.566847,2.223392
3,2024-11-07,5973.100098,19362.51953,3470.660889,4.558451,1.121118,4.847245
4,2024-11-08,5995.540039,19215.48047,3452.297119,4.951259,0.353203,4.292483
...,...,...,...,...,...,...,...
130,2025-05-06,5606.910156,23249.65039,3316.114014,-1.851663,21.421731,0.178448
131,2025-05-07,5631.279785,23115.96094,3342.665039,-1.425076,20.723535,0.980543
132,2025-05-08,5663.939941,23352.68945,3351.996094,-0.853363,21.959854,1.262430
133,2025-05-09,5659.910156,23499.32031,3341.999023,-0.923904,22.725637,0.960423


In [251]:



#open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
sh = gc.open('TrumptrackerRaw')

#select the first sheet 
wks = sh[0]

wks.set_dataframe(since_inauguration,(0,0))


In [253]:
from datetime import datetime

# Get the current year
current_year = datetime.now().year
print(current_year)

2025


In [254]:
df=pd.DataFrame()
for year in range(2024,current_year+1):
    print(year)
    url=f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/{year}/all?type=daily_treasury_yield_curve&field_tdr_date_value={year}&page&_format=csv"
    response = requests.get(url)

    file_object = io.StringIO(response.content.decode('utf-8'))
    data=pd.read_csv(file_object)
    df=pd.concat([df,data],axis=0)
df

2024
2025


,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,1.5 Month
0,12/31/2024,4.40,4.39,4.37,4.32,4.24,4.16,4.25,4.27,4.38,4.48,4.58,4.86,4.78,NaN
1,12/30/2024,4.43,4.42,4.37,4.33,4.25,4.17,4.24,4.29,4.37,4.46,4.55,4.84,4.77,NaN
2,12/27/2024,4.44,4.43,4.31,4.35,4.29,4.20,4.31,4.36,4.45,4.53,4.62,4.89,4.82,NaN
3,12/26/2024,4.45,4.45,4.35,4.37,4.31,4.23,4.30,4.35,4.42,4.49,4.58,4.83,4.76,NaN
4,12/24/2024,4.44,4.44,4.40,4.38,4.30,4.24,4.29,4.36,4.43,4.52,4.59,4.84,4.76,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,01/08/2025,4.41,4.34,4.35,4.31,4.25,4.19,4.28,4.31,4.45,4.56,4.67,4.97,4.91,NaN
86,01/07/2025,4.42,4.35,4.35,4.31,4.24,4.19,4.30,4.33,4.46,4.57,4.67,4.97,4.91,NaN
87,01/06/2025,4.43,4.36,4.35,4.31,4.24,4.17,4.28,4.30,4.42,4.52,4.62,4.91,4.85,NaN
88,01/03/2025,4.44,4.35,4.34,4.31,4.25,4.18,4.28,4.32,4.41,4.51,4.60,4.88,4.82,NaN


In [255]:
df["Date"]=pd.to_datetime(df["Date"])
since_inauguration_yield=df[df["Date"]>="2024-11-04"]
since_inauguration_yield=since_inauguration_yield[["Date","10 Yr"]]
since_inauguration_yield=since_inauguration_yield.sort_values(by="Date", ascending=True)
since_inauguration_yield

,Date,10 Yr
38,2024-11-04,4.31
37,2024-11-05,4.26
36,2024-11-06,4.42
35,2024-11-07,4.31
34,2024-11-08,4.30
...,...,...
4,2025-05-06,4.30
3,2025-05-07,4.26
2,2025-05-08,4.37
1,2025-05-09,4.37


In [256]:



#open the google spreadsheet (where 'PY to Gsheet Test' is the name of my sheet)
sh = gc.open('TrumptrackerRaw')

#select the first sheet 
wks = sh[9]

wks.set_dataframe(since_inauguration_yield,(0,0))
